The following code is best run with Python 3.7 or ealier. Other versions of python might create conflicts with key libaries needed to run GECToR

In [1]:
#getting RoBERTa and GECToR. Check the directory to make sure everything is in the correction location
!git clone https://github.com/grammarly/gector.git
!wget https://grammarly-nlp-data-public.s3.amazonaws.com/gector/roberta_1_gectorv2.th
!mv roberta_1_gectorv2.th ./gector
!cd ./gector

fatal: destination path 'gector' already exists and is not an empty directory.
--2023-05-03 22:20:13--  https://grammarly-nlp-data-public.s3.amazonaws.com/gector/roberta_1_gectorv2.th
Resolving grammarly-nlp-data-public.s3.amazonaws.com (grammarly-nlp-data-public.s3.amazonaws.com)... 54.231.135.137, 52.217.110.204, 54.231.135.225, ...
Connecting to grammarly-nlp-data-public.s3.amazonaws.com (grammarly-nlp-data-public.s3.amazonaws.com)|54.231.135.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 514093595 (490M) [binary/octet-stream]
Saving to: ‘roberta_1_gectorv2.th’

roberta_1_gectorv2. 100%[===================>] 490,28M  12,4MB/s    in 41s     

2023-05-03 22:20:55 (11,9 MB/s) - ‘roberta_1_gectorv2.th’ saved [514093595/514093595]



In [6]:
!pip install -r gector/requirements.txt

The following cell allows you to test and play around with GECToR. You can replace the variable 'sent' with a grammartically incorrect sentence to check how well it works. 

In [38]:

from gector.gector.gec_model import GecBERTModel

# Create an instance of the model
model = GecBERTModel(vocab_path = "./gector/data/output_vocabulary", model_paths = ["./gector/roberta_1_gectorv2.th"])

# Add the sentence with grammatical errors
sent = 'I and my friend is going to the park'

# Define a function to correct grammatical errors of a given sentence
def correction(sent):
    batch = []
    batch.append(sent.split())
    final_batch, total_updates = model.handle_batch(batch)
    updated_sent = " ".join(final_batch[0])
    return updated_sent

print(f"Original Sentence: {sent}\n")
print(f"Updated Sentence: {correction(sent=sent)}")

Original Sentence: I and my friend is going to the park

Updated Sentence: We went to the park .


In [25]:
import pandas as pd
df = pd.read_csv("testing_data_cleaned.csv")
# df
# fill in the values of col5 into col6 
df['col6'].fillna(df['col5'], inplace=True)
df

,col5,col6
0,Wan na Learn English ?,Wan na Learn English ?
1,Not that much .,Not that much .
2,"A family , seemingly a father , a mother and t...","A family , seemingly a father , a mother and t..."
3,"A funky music starts , singing some worst obsc...","Funky music starts , playing some bad obscenit..."
4,It seems they do n't care so much about its ly...,It seems they do n't care so much about its ly...
...,...,...
9214,but today I went by foot so it rained .,but today I went on foot because it rained .
9215,The weather forecast says that it will rain to...,The weather forecast says that it will rain to...
9216,It is depressing .,It is depressing .
9217,Deaflympic,Deaflympic


In [29]:
# iterate through the sentences
tp = tn = fp = fn = 0
for i, sent in enumerate(df.col5) :
    corr = correction(sent)
    if corr == sent :
         tn +=1 

    else :
        if sent != df.col6[i] :
            fn += 1
        else :
            fp +=1

tp = df.shape[0] - fn - tn

print('True Positive', tp)
print('True Negative', tn)
print('False Positive', fn)
print('False Positive', fp)

True Positive 1395
True Negative 4696
False Positive 3128
False Positive 1395


In [30]:
precision = tp/(tp+fp)
recall = tp/(tp+fn)
F = (1+0.5**2)*((precision*recall)/((0.5**2 * precision)+recall))
F

0.4447490913728241